In [ ]:
import datetime as dt
import fnmatch
import os
import time
from io import StringIO

import geopandas as gpd
import pandas as pd
import requests
from rasterio.errors import RasterioIOError

import seabeepy as sb
from seabeepy.config import SETTINGS

In [ ]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

GEONODE_URL = os.environ.get("GEONODE_URL", r"https://geonode.seabee.sigma2.no/api/v2")

# Fix datasets with no abstract

Updating metadata via the GeoNode API occasionally times-out and fails. This leads to datasets being published correctly, but without any abstract information. This notebook attempts to identify datasets where this has happened and deletes them for republishing.

## 1. Identify datasets with no abstract

In [ ]:
def get_geonode_data(geonode_url):
    # Initialize an empty list to store each dataset's information
    datasets = []

    # Initialize the URL for the first page of results
    url = f"{geonode_url}/datasets/"

    while url:
        # Get a page of results
        response = requests.get(url)
        data = response.json()

        for dataset in data["datasets"]:
            datasets.append(
                {
                    "name": dataset["name"],
                    "id": dataset["pk"],
                    "type": dataset["subtype"],
                    "abstract": dataset["abstract"],
                }
            )

        # Get the URL for the next page of results, or None if this is the last page
        url = data["links"]["next"]

        time.sleep(0.5)

    # Convert the list into a GeoDataFrame
    df = pd.DataFrame(datasets)

    return df


def find_file(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for filename in fnmatch.filter(files, pattern):
            return os.path.join(root, filename)
    return None

In [ ]:
# Get all missions
df = get_geonode_data(GEONODE_URL)

# Missions with no abstract
noabs_df = df[df["abstract"].str.contains("No abstract")].reset_index(drop=True)
print(len(noabs_df), "datasets without an abstract.")
noabs_df.head()

## 2. Fix datasets with no abstract

On both MinIO and GeoNode.

In [ ]:
# Will only delete stuff if 'dry_run' is False
dry_run = False
det_pub_thresh = 0.7

In [ ]:
def delete_dataset(ds_id, file_path, file_name):
    del_url = f"{GEONODE_URL}/resources/{ds_id}"
    response = requests.request("DELETE", del_url, auth=auth)
    response.raise_for_status()
    sb.storage.delete_file(file_path, minio_client)
    print(f"Deleted: {file_name}")


def update_vector_dataset_abstract(mission_dir):
    mission_name = sb.ortho.get_layer_name(mission_dir)
    res_dir = sb.ml.get_latest_results_dir(mission_dir)
    ml_options = sb.ml.get_ml_options(mission_dir)
    if ml_options["task"] == "detection":
        res_path = os.path.join(res_dir, "out.gpkg")
        gdf = gpd.read_file(res_path)
        layer_name = f"{mission_name}_all-detections"
        layer_name_filt = f"{mission_name}_detections"
        gdf = sb.ml.convert_seabird_class_codes_to_names(
            gdf, SETTINGS.DB_USER, SETTINGS.DB_PASSWORD
        )
        gdf_filt = gdf.query("score_species >= @det_pub_thresh").copy()
        style_dict = {
            layer_name_filt: f"red_outline.sld",
        }
        metadata = {
            "abstract": sb.geo.get_detection_abstract(
                gdf_filt,
                layer_name_filt,
                ml_options["model"],
                str(gpkg_path),
                det_pub_thresh,
            ),
            "date": dt.datetime.strptime(
                gdf_filt["datetimereg"].iloc[0], "%Y-%m-%d %H:%M:%S"
            ).isoformat(),
            "date_type": "creation",
            "attribution": "SeaBee",
        }
        if not dry_run:
            sb.geo.update_geonode_metadata(
                layer_name_filt,
                SETTINGS.GEONODE_USER,
                SETTINGS.GEONODE_PASSWORD,
                metadata,
            )
        else:
            print(metadata)

## 2a. Delete raster datasets with no abstract

The code cell below deletes raster datasets on the GeoNode without an abstract. These will be picked up by the pipeline and standardised again, followed by republishing (hopefully with full metadata).

In [ ]:
# Delete datasets without an abstract (on both GeoNode and MinIO)
base_fold = r"/home/notebook/shared-seabee-ns9879k"
auth = (SETTINGS.GEONODE_USER, SETTINGS.GEONODE_PASSWORD)

cnt = 0
skip = 0
for idx, row in noabs_df.iterrows():
    name = row["name"]
    ds_id = row["id"]
    ds_type = row["type"]

    if ds_type == "raster":
        tif_name = f"{name}.tif"
        tif_path = find_file(base_fold, tif_name)
        if tif_path:
            if not dry_run:
                delete_dataset(ds_id, tif_path, tif_name)
            else:
                print(f"Would delete: {tif_name}")
            cnt += 1
        else:
            print(f"Not found: {tif_name}")

    # elif ds_type == "vector":
    #     ml_type = name.split("_")[-1]
    #     if ml_type == "detections":
    #         # Seabirds or mammals
    #         gpkg_name = f"{name}.gpkg"
    #         gpkg_path = find_file(base_fold, gpkg_name)
    #         if gpkg_path:
    #             if not dry_run:
    #                 delete_dataset(ds_id, gpkg_path, gpkg_name)
    #             else:
    #                 print(f"Would delete: {gpkg_name}")
    #             cnt += 1
    #         else:
    #             print(f"Not found: {gpkg_name}")

    #     elif ml_type == "classifications":
    #         # Habitats
    #         print(f"Fixing classifications not yet implemented: {name}")
    #     else:
    #         skip += 1
    #         print(f"Skipping: {name}")

print(f"\n{skip} datasets skipped; {cnt} datasets deleted.")

## 2b. Update abstract for vector datasets

The code cell below attempts to update the metadata for vector layers with no abstract. Note that this will only be successful if the corresponding raster is already published with full metadata. If the raster was also missing metadata, first run the code cell above and wait for the raster to be republished before running this code.

In [ ]:
base_fold = r"/home/notebook/shared-seabee-ns9879k/seabirds/2025"
for idx, row in noabs_df.iterrows():
    try:
        name = row["name"]

        if "2025" not in name:
            continue
        gpkg_path = find_file(base_fold, f"{name}.gpkg")
        if gpkg_path:
            print(f"\nUpdating {name}")
            mission_dir = gpkg_path.split("results")[0]
            try:
                update_vector_dataset_abstract(mission_dir)
            except ValueError as e:
                print(e)
            time.sleep(1)
    except RasterioIOError as e:
        print("ERROR:")
        print(str(e))
        print("You probably need to wait for this raster to be republished.")
        continue